Copyright (c) Alliance for Sustainable Energy, LLC and Skye Analytics, Inc. See also https://github.com/NREL/ATB-calc/blob/main/LICENSE

# Run ATB LCOE pipeline
This Jupyter notebook is an example of how to run the ATB data processing pipeline and extract data from the ATB electricity data workbook. The path to the workbook on your computer needs to be updated as indicated below. The notebook will otherwise run as-is. There is commented-out code in several locations showing alternative ways to run the pipeline. 

In [ ]:
import os
import sys
import pandas as pd
from datetime import datetime as dt

sys.path.insert(0, os.path.dirname(os.getcwd()))
from lcoe_calculator.process_all import ProcessAll
from lcoe_calculator.tech_processors import (ALL_TECHS,
    FixedOffShoreWindProc, FloatingOffShoreWindProc, LandBasedWindProc, DistributedWindProc,
    UtilityPvProc, CommPvProc, ResPvProc, UtilityPvPlusBatteryProc,
    CspProc, GeothermalProc, HydropowerProc, PumpedStorageHydroProc,
    PumpedStorageHydroOneResProc,
    CoalProc, NaturalGasProc, NuclearProc, BiopowerProc,
    UtilityBatteryProc, CommBatteryProc, ResBatteryProc,
    CoalRetrofitProc, NaturalGasRetrofitProc, NaturalGasFuelCellProc)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Load the ATB data workbook & prepare for processing
Update the path for `atb_electricity_workbook` to point to your copy of the ATB electricity data
workbook downloaded from the [ATB website](https://atb.nrel.gov). 

The pipeline can be run for one, several, or all ATB electricity technologies. 


In [ ]:
# The below line MUST be updated to reflect the location of the ATB workbook on your computer
atb_electricity_workbook = 'path/to/2024_v1_Workbook_06_25_24.xlsx'

# ---- Comment/uncomment the below lines to process all techs or a subset of techs
# Process all technologies
techs = ALL_TECHS

# Process a couple technologies
techs = [LandBasedWindProc, UtilityPvProc]

# Or process a single technology
techs = LandBasedWindProc

# Initiate the processor with the workbook location and desired technologies
processor = ProcessAll(atb_electricity_workbook, techs)

## Run the pipeline
Now that the processor knows where the data workbook is and which technologies we are interested in, we
can kick it off. Depending on the number of requested technologies, this can take a couple minutes.
Note that calculated LCOE and CAPEX is automatically compared to the values in the workbook. Not all
technologies have LCOE and CAPEX.

In [ ]:
start = dt.now()
processor.process()
print('Processing completed in ', dt.now() - start)

## Data extraction
Congratulations! The pipeline has run and the data is available. Several options for extracting the code are shown below. 

In [ ]:
# Save data to as a CSV
processor.to_csv('atb_data.csv')

# Save flattened data to as a CSV
processor.flat_to_csv('atb_data_flat.csv')

# Save meta data to as a CSV
processor.meta_data_to_csv('atb_meta_data.csv')

# Copy data to the clipboard so it can be pasted in a spreadsheet 
processor.data.to_clipboard()

## Exploring the data in the notebook
The extracted and calculated parameters can also be viewed directly in the notebook if desired. Note that the data is a combination of all technologies that were processed and may need to be filtered. Working with a single technology processor as described in the `Tech Processor.ipynb` notebook may be easier depending on your goals. Data is internally stored as a [pandas](https://pandas.pydata.org) data frame.

In [ ]:
data = processor.data

# Show available parameters
print('Available parameters')
display(data.Parameter.unique())

# Display LCOE
print('Calculated LCOE')
display(data[data.Parameter == "LCOE"])

# Display capacity factor
print('Capacity factor')
display(data[data.Parameter == "CF"])